In [1]:
import numpy as np
import cv2
import task1
import matplotlib.pyplot as plt
import copy
import json

In [30]:
img = cv2.imread("./data/proj1-task2.jpg")#,cv2.IMREAD_GRAYSCALE)
b_try = cv2.imread("./data/b_try.jpg")# cv2.IMREAD_GRAYSCALE);

In [3]:
def show_image(img, delay=10000):
    cv2.imshow('IMG',img)
    cv2.waitKey(delay)
    cv2.destroyAllWindows()

In [4]:
# a_template = img[136:146, 240:248]
# c_template = img[215:223, 122:129]
b_template = img[185:200,95:102]

In [23]:
b_template = img[158:173,211:217]
show_image(b_template,0)

In [6]:
# b_template[b_template>220] = 255
# b_template[b_template<200] = 130
# b_template[b_template<150] = 100
# b_template[b_template<90] = 70

In [7]:
show_image(img, 0)

In [8]:
show_image(b_template,0)

In [8]:
# cv2.imwrite('./data/a.jpg', a_template)
# cv2.imwrite('./data/b.jpg', b_template)
# cv2.imwrite('./data/c.jpg', c_template)

In [15]:
def templateMatching(img, template, threshold):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    w, h = template.T.shape
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    loc = np.where( res >= threshold)
    cnt = 0
    for pt in zip(*loc[::-1]):
        cv2.rectangle(img, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        cnt+=1
#     print(cnt)
    ##temp
    return cnt
    return img

# cv2.imwrite('res.png',img_rgb)

In [16]:
def show_matches(img, coordinates, template, delay = 0):
    img = img.copy()
    w,h = len(template[0]), len(template)
    for x,y in coordinates:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,0,255), 1)
    show_image(img, delay)

In [17]:
#Reference - https://docs.opencv.org/2.4/doc/tutorials/imgproc/histograms/template_matching/template_matching.html
def get_ccoeff_normed(img, template):
    
    arr_element_sub = lambda arr, val: [arr_val - val for arr_val in arr]
    element_squared_sum = lambda arr: sum(z**2 for z in arr)
    sum_normed = lambda arr, const: sum(arr) / const
    sum_of_prods = lambda arr1, arr2: sum([a1_val * a2_val for a1_val, a2_val in zip(arr1, arr2)])
    flatten = lambda arr : [arr_val for arr_row in arr for arr_val in arr_row]

    w ,h =  len(template), len(template[0]),
    wh_prod = w*h*1.0
    
    ccoeff_normed = [[0.0 for _ in range(len(img[0]) - h + 1)] for _ in range(len(img) - w + 1)]
#     print("w %d h %d ccnormed_len %d ccnormed_len %d" %(w,h, len(ccoeff_normed),len(ccoeff_normed[0])))

    #Is same for every sliding window as template doesn't change
    template = flatten(template)
    t_sum_normed = sum_normed(template, wh_prod)
    t_dash = arr_element_sub(template, t_sum_normed)
    t_dash_sum_squared = element_squared_sum(t_dash)

    #Hack for zeros
    if(t_dash_sum_squared == 0):
        t_dash_sum_squared = 1e-6
#     print('t_sum_normed %f t_dash %s t_dash_sum_sq %d'%(t_sum_normed, t_dash, t_dash_sum_squared))

    def get_ccoeff_normed_val(i, j,):
#         nonlocal img, template
#         nonlocal t_sum_normed

        #Flatten the image
        im = [img[x][y] for x in range(i, i + w) for y in range(j, j + h)]
#         print('Im',im)

        #changes with the window
        #check for indices
        i_sum_normed = sum_normed(im, wh_prod)
        i_dash = arr_element_sub(im, i_sum_normed)
        i_dash_sum_squared = element_squared_sum(i_dash)
        if i_dash_sum_squared == 0:
            i_dash_sum_squared = 1e-6
    
        
#         print('i_sum_normed %d \nimg = %s\n i_dash %s\n i_dash_sum_sq %d'%(i_sum_normed, im, i_dash, t_dash_sum_squared))        
#         print('t_Val=%d i_val=%d'%(len(t_dash), len(i_dash)))
#         input()
        
        numerator = sum_of_prods(t_dash, i_dash)
        denominator = np.sqrt(t_dash_sum_squared * i_dash_sum_squared)
        
#         print(numerator, denominator)
        return numerator/denominator
    
    for x in range(len(ccoeff_normed) - w):
        for y in range(len(ccoeff_normed[0]) - h):
            ccoeff_normed[x][y] = get_ccoeff_normed_val(x, y)
    return ccoeff_normed

In [18]:
def to_grayscalelist(img):
    if type(img)==list:
        return cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY).tolist()
    else:
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).tolist()

In [19]:
def detect(img, template, threshold=0.7):
    ccoeff_normed_mat = get_ccoeff_normed(img, template)
    #Get the coordinates
    #Inverted as the x maps to columns and y maps to rows from pixel coords to matrix
    coordinates = [(y,x) for x in range(len(ccoeff_normed_mat)) \
        for y in range(len(ccoeff_normed_mat[0])) if ccoeff_normed_mat[x][y] >= threshold]    
    return coordinates


In [20]:
def detect_and_show(img, template, threshold=0.7):
    coords =detect(to_grayscalelist(img), to_grayscalelist(template), threshold)
    show_matches(img, coords, template)
    return coords

In [21]:
# a_coordinates = detect_and_show(img, a_template, 0.62)

In [36]:
b_coordinates = detect_and_show(img, b_try, 0.74)

In [ ]:
# c_coordinates = detect_and_show(img, c_template, 0.75)

In [38]:
def plot_and_show(img, alphabet, thickness = 2):
    dictionary = json.load(open('temp/%s.json' % alphabet))
    coordinates = dictionary['coordinates']
    template_size = dictionary['templat_size']
    img = img.copy()
    h,w = template_size
    for x,y in coordinates:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), thickness)
    show_image(img, 0)

In [42]:
plot_and_show(img, 'b',1 )